In [3]:
import pandas as pd
import numpy as np
from collections import Counter

In [4]:

def check(a1, a2, verbose=False):
    def explain(*args,verbose=verbose):
        if verbose:
            print(*args)
    """' check whether two dataframes are one-to-one , many-to-one"""
    set1 = set(a1)
    key1_is_unique= ( len(set1) == len(a1))
    set2 = set(a2)
    key2_is_unique = (len(set2) == len(a2))
    two_sets_equal = (set1 == set2)
    if two_sets_equal:
        #key1 and key2 unique values are the same"
        if key1_is_unique and key2_is_unique:    
            explain("One-to-one, key1 and key2 match") 
            return "1-to-1"   
        elif not key1_is_unique and not key2_is_unique:
            explain("Many-to-many, all values in both, but both have duplicates")
            return "n-to-n"
        elif key1_is_unique and not key2_is_unique:
            explain("One-to-many, all values in both, key2 is facts (has duplicates) , key1 is the dimension/master table")
            return "1-to-n"
        elif key2_is_unique and not key1_is_unique:
            explain( "Many-to-one, all values in both, key1 is facts (has duplicates), key2 is dimension/master table")
            return "n-to-1"
    else: #two sets are not equal, we need to find out how so
        intersection=set1.intersection(set2)
        if len(intersection)==0: #Disjoint sets, no commonalities
            if key1_is_unique:
                if key2_is_unique:
                    explain("No common values, and both have no duplicates")
                    return "disjoint - no duplicates"
                else:
                    explain("No common values, key1 has no duplicates, key2 has duplicates")
                    return "disjoint - duplicates in key2"
            else:
                if key2_is_unique:
                    explain("No common values, key2 has no duplicate values, key1 has duplicates")
                    return "disjoint - duplicates in key1"
                else:
                    explain( "No common values, both have duplicates also in the data")
                    return "disjoint - both have duplicates"
        else: #intersection is not null, so there are some common elements
            set1_in_set2=set1.issubset(set2)
            set2_in_set1=set2.issubset(set1)
            count1=Counter(a1)
            count2=Counter(a2)
            if set1_in_set2:
                if key2_is_unique:
                    explain("Many-to-one, key2 is dimension (no duplicates) but has values not in key1")
                    return "*-to-1"
                else: #key2 has duplicates, so it is likely a fact table
                    if key1_is_unique:
                        explain("key1 is possible dimension (no duplicates) but misses values in key2")
                        return "1-to-* - need fix incomplete key1"
                    else:
                        explain("key1 and key2 have duplicates, key1 is likely facts as is subset of key2")
                        return "*-to-1 - need fix duplicate keys in key2"
            if set2_in_set1:
                if key1_is_unique:
                    explain("One-to-Many, key1 is dimension, no duplicates but has values not in key2")
                    return "1-to-*"
                else:
                    if key2_is_unique:
                        explain("key2 is possible dimension (no duplicates) but misses values in key1")
                        return "*-to-1 - need fix incomplete key2"
                    else:
                        explain("key1 and key2 have duplicates, key2 is likely facts as is subset of key1")
                        return "1-to-* - need fix duplicate keys in key1"
            explain("Both key1 and key2 have values not shared between them") 
            set1diffset2=set1-set2
            set2diffset1=set2-set1
            if key1_is_unique:
                if key2_is_unique:
                    explain("Both have no duplicates values.\nThey share", len(intersection)," unique values.\nKey1 has ",len(set1diffset2)," unique values non shared.\nKey2 has ",len(set2diffset1)," unique values non shared")
                    return "partial overlap and no duplicates in either"
                else:
                    key2_all_non_shared=[x for x in a2 if x in set2diffset1]
                    key2_all_non_shared_dup=[x for x in a2 if ((count2[x]>1) and (x in set2diffset1))]
                    explain("Key1 has unique values but key2 has duplicates.\nThey share", len(intersection)," unique values.\nKey1 has ",len(set1diffset2)," unique values non shared.\nKey2 has ",len(set2diffset1)," unique values non shared.\nKey2 has ",len(key2_all_non_shared)," elements non shared.\nKey2 has ", len(key2_all_non_shared_dup)," elements duplicated and non shared")

                    return "partial overlap and key2 has duplicates"

            else:
                if key2_is_unique:
                    key1_all_non_shared=[x for x in a1 if x in set1diffset2]
                    key1_all_non_shared_dup=[x for x in a1 if ((count1[x]>1) and (x in set1diffset2))]
                    explain("Key2 has unique values but key1 has duplicates.\nThey share", len(intersection)," unique values.\nKey1 has ",len(set1diffset2)," unique values non shared.\nKey2 has ",len(set2diffset1)," unique values non shared.\nKey1 has ",len(key1_all_non_shared)," elements non shared.\nKey1 has ", len(key1_all_non_shared_dup)," elements duplicated and non shared")

                    return "partial overlap and key1 has duplicates"
                else:
                    explain("Both have duplicates and values non shared with each other")

                    return "partial overlap and both have duplicates"



                


In [5]:
A =  [10, 20, 30, 40, 50]
Ax = [10,10, 20, 30, 40, 50]
B =  [10, 20, 30, 40, 50, 60,70]
Bx = [10,10, 20, 30, 40, 50, 60,60,60,60]
By=  [10,20,30,40,50,50,60,60,70,70]
C =  [10, 20, 30, 40, 50]
Cx = [10,10, 30, 20, 40, 50]
D =  [100, 200, 300, 400, 500]
Dx=  [100,200,300,400,400,500,500]
E =  [10, 20, 60, 70, 80, 90]
Ex=  [10,20,60,60,70,80,90,90,90]
assert check(A,C)== "1-to-1"
assert check(C,A)== "1-to-1"
assert check(A,B) == "*-to-1"
assert check(B,A) == "1-to-*"
assert check(B,By)=="1-to-n"
assert check(A,Cx)=="1-to-n"
assert check(By,B)=="n-to-1"
assert check (Ax,Ax)=="n-to-n"
assert check (Ax,Bx)=='*-to-1 - need fix duplicate keys in key2'
assert check(Bx,Ax)=='1-to-* - need fix duplicate keys in key1'
assert check(A,Bx)=='1-to-* - need fix incomplete key1'
assert check(Bx,A)=='*-to-1 - need fix incomplete key2'
assert check(A,E)=='partial overlap and no duplicates in either'
assert check(A,Ex)=='partial overlap and key2 has duplicates'
assert check(Ex,A)=='partial overlap and key1 has duplicates'
assert check(Ex,Ax)=='partial overlap and both have duplicates'
assert check(A,D)== "disjoint - no duplicates"
assert check(D,A)== "disjoint - no duplicates"
assert check(A,Dx)=="disjoint - duplicates in key2"
assert check(Dx,A)=="disjoint - duplicates in key1"
assert check(Dx,Ax)== "disjoint - both have duplicates"
assert check(Ax,Dx)== "disjoint - both have duplicates"
assert check(D,Dx)=="1-to-n"

In [18]:
n=1000000
n80=int(n*.8)
n50=int(n*.5)
n20=int(n*.2)
n150=int(n*1.5)
n200=int(n*2)
A=list(range(1,int(n)))
B=list(range(1,n200))
Ax=A+A
Bx=list(range(n20,n150))+B
By=B+B
C=list(range(1,int(n)))
Cx=C+C[n20:n80]
D=list(range(n+1,n200))
Dx=D+D[1:n50]+D[n50:n80]
E=list(range(n80,n))+list(range(n150,n200))
Ex=E+list(range(n150,n200))


In [19]:
assert check(A,C)== "1-to-1"
assert check(C,A)== "1-to-1"
assert check(A,B) == "*-to-1"
assert check(B,A) == "1-to-*"
assert check(B,By)=="1-to-n"
assert check(A,Cx)=="1-to-n"
assert check(By,B)=="n-to-1"
assert check (Ax,Ax)=="n-to-n"
assert check (Ax,Bx)=='*-to-1 - need fix duplicate keys in key2'
assert check(Bx,Ax)=='1-to-* - need fix duplicate keys in key1'
assert check(A,Bx)=='1-to-* - need fix incomplete key1'
assert check(Bx,A)=='*-to-1 - need fix incomplete key2'
assert check(A,E)=='partial overlap and no duplicates in either'
assert check(A,Ex)=='partial overlap and key2 has duplicates'
assert check(Ex,A)=='partial overlap and key1 has duplicates'
assert check(Ex,Ax)=='partial overlap and both have duplicates'
assert check(A,D)== "disjoint - no duplicates"
assert check(D,A)== "disjoint - no duplicates"
assert check(A,Dx)=="disjoint - duplicates in key2"
assert check(Dx,A)=="disjoint - duplicates in key1"
assert check(Dx,Ax)== "disjoint - both have duplicates"
assert check(Ax,Dx)== "disjoint - both have duplicates"
assert check(D,Dx)=="1-to-n"